### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

In [4]:
!{sys.executable} -m pip install geopandas

In [5]:
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [6]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [7]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [8]:
import geopandas as gpd
import glob
import os

### Verificar que están localmente los archivos Shapefile. Si no están, descargarlos

In [9]:
if not os.path.exists('Shapefiles'):
    bucket_shapefiles = 'iadbprod-csd-hub-analyticaldata'
    folder_local = 'Shapefiles'
    os.mkdir(folder_local)
    for key in s3.Bucket(bucket_shapefiles).objects.all().filter(Prefix=folder_local):
        data_key = key.key
        directorios = data_key.split('/')
        if len(directorios) == 3 and not 'Municipios' in data_key: # carpeta originen, pais y archivo (shp, cpg, csv, dbf, prj, shx)
            folder_pais = os.path.join(folder_local, directorios[1])
            if not os.path.exists(folder_pais):
                os.mkdir(folder_pais)
            new_data_key = os.path.join(folder_pais,directorios[2])
            s3.Bucket(bucket_shapefiles).download_file(data_key, new_data_key)

### Seleccionar shapefiles de paises especificos
##### borrar carpetas en terminal / rm -rf "shapefiles2"

### Crea primera tabla historica preliminar a nivel de hogares 

In [10]:
# Tabla historica preliminar de hogares
fecha_inicio = '02-01-2020'
fecha_final = '02-29-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)
fechas

DatetimeIndex(['2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
               '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
               '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12',
               '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16',
               '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20',
               '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
               '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28',
               '2020-02-29'],
              dtype='datetime64[ns]', freq='D')

In [11]:
historico_hogares_deltas = f'historico_hogares_preliminar'
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    tabla_usuarios_fecha = f'todos_usuarios_delta_{codigo_fecha}'
    if i == 0 and (day == '01' and fecha.month == 2):
        query = f'''
                    CREATE TABLE {historico_hogares_deltas}
                    WITH ( external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_hogares/Tablas_hogares_preliminar/{historico_hogares_deltas}', 
                        format = 'PARQUET', 
                        parquet_compression = 'SNAPPY') AS
                    SELECT caid, lat_hogar, long_hogar, n_obs_hogar, iso_country_code
                    FROM {tabla_usuarios_fecha} 
                    WHERE lat_hogar IS NOT NULL
                '''
    else:
        query = f'''
                    SELECT caid, lat_hogar, long_hogar, n_obs_hogar, iso_country_code FROM {tabla_usuarios_fecha} WHERE lat_hogar IS NOT NULL
                    UNION
                    SELECT caid, lat_hogar, long_hogar, n_obs_hogar, iso_country_code FROM {historico_hogares_deltas} WHERE lat_hogar IS NOT NULL
                '''
    cursor.execute(query)
    tiempo = (time.time()-time_fecha)/60
    print(f"{codigo_fecha} insertada, {int(tiempo)} minutos")
    time.sleep(10)
print((time.time()-start_time)/60)

201
201 insertada, 0 minutos
202
202 insertada, 1 minutos
203
203 insertada, 1 minutos
204
204 insertada, 1 minutos
205
205 insertada, 1 minutos
206
206 insertada, 1 minutos
207
207 insertada, 1 minutos
208
208 insertada, 1 minutos
209
209 insertada, 1 minutos
210
210 insertada, 1 minutos
211
211 insertada, 1 minutos
212
212 insertada, 1 minutos
213
213 insertada, 1 minutos
214
214 insertada, 1 minutos
215
215 insertada, 1 minutos
216
216 insertada, 1 minutos
217
217 insertada, 1 minutos
218
218 insertada, 1 minutos
219
219 insertada, 1 minutos
220
220 insertada, 1 minutos
221
221 insertada, 1 minutos
222
222 insertada, 1 minutos
223
223 insertada, 1 minutos
224
224 insertada, 1 minutos
225
225 insertada, 1 minutos
226
226 insertada, 1 minutos
227
227 insertada, 1 minutos
228
228 insertada, 1 minutos
229
229 insertada, 1 minutos
36.64538665612539


In [12]:
query = f'''MSCK REPAIR TABLE {historico_hogares_deltas}'''
cursor.execute(query)

### Primera asignación de regiones a hogares

In [13]:
# Fecha de primera tabla a nivel de hogares
fecha_inicio = '02-01-2020'
fecha_final = '02-01-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)
fechas

DatetimeIndex(['2020-02-01'], dtype='datetime64[ns]', freq='D')

In [14]:
historico_hogares_deltas = f'historico_hogares_preliminar'
for fecha in fechas:
    start_time_day = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    
    
    # Generando tabla temporal de coordenadas de hogates
    # ==============================
    todos_usuarios_delta = f'todos_usuarios_delta_dist_corregida_{codigo_fecha}'
    print(todos_usuarios_delta)
    starttime=time.time()
    query = f'''SELECT delta.lat_final as lat_hogar,
                       delta.long_final as long_hogar,
                       delta.iso_country_code  
                FROM (SELECT filtro.caid, 
                             filtro.n_obs, 
                             filtro.lat_final, 
                             filtro.long_final, 
                             filtro.year, 
                             filtro.month, 
                             filtro.day, 
                             filtro.iso_country_code 
                     FROM {todos_usuarios_delta} AS filtro 
                     LEFT JOIN (SELECT ROUND(lat_hogar,3) AS lat_hogar2, 
                                       ROUND(long_hogar,3) AS long_hogar2, 
                                       iso_country_code 
                                FROM {historico_hogares_deltas} 
                                GROUP BY ROUND(lat_hogar,3),
                                         ROUND(long_hogar,3),
                                         iso_country_code) AS t2  
                    ON filtro.lat_final=t2.lat_hogar2 
                    AND filtro.long_final=t2.long_hogar2 
                    AND filtro.iso_country_code=t2.iso_country_code) AS delta 
                GROUP BY delta.lat_final,
                         delta.long_final,
                         delta.iso_country_code'''
    df_usuario_coord = cursor.execute(query).as_pandas()    
    print(f"Tiempo query select: {int((time.time()-starttime))}s")
    print(f"Tamaño tabla sin id_provincia en historico: {df_usuario_coord.shape[0]}")
    
    
    # Asignando region a la tabla hogares por dia
    # ==============================
    starttime=time.time()
    paises = ['AR','CO','PE','NI','SV','BO','DO','CL','CR','PY','BZ','HN','JM','TT','UY','GY','EC','GT','PA','VE',"BR","MX"]
    for pais in paises:
        print(f"Pais: {pais}")
        df_usuario_coord2 = df_usuario_coord.loc[df_usuario_coord["iso_country_code"]==pais]
        print(df_usuario_coord2.shape)
        gdf_homes = gpd.GeoDataFrame(df_usuario_coord2,
                                     geometry=gpd.points_from_xy(
                                         df_usuario_coord2.long_hogar, 
                                         df_usuario_coord2.lat_hogar), 
                                         crs = 'epsg:4326')
        gdf_homes = gdf_homes.dropna(subset=['long_hogar','lat_hogar'])
        path = glob.glob(f'Shapefiles/{pais}/*.shp')
        mapa = gpd.read_file(path[0],encoding = 'utf-8')
        mapa = mapa.to_crs("epsg:4326")
        if pais == "CL":
            mapa = mapa.loc[:,['codregion', 'Region', 'geometry']]
            gdf_homes = gpd.sjoin(gdf_homes, mapa, op='within') 
            gdf_homes=gdf_homes.loc[:,['long_hogar','lat_hogar',"codregion", 'Region','iso_country_code']]
            gdf_homes=gdf_homes.rename(columns={"codregion":"id_provincia",'Region': 'Provincia'})
            gdf_homes=gdf_homes.dropna(subset=["id_provincia"])
        if pais == "CR":       
            mapa = mapa.loc[:,['COD_PROV', 'NPROVINCIA', 'geometry']]
            gdf_homes = gpd.sjoin(gdf_homes, mapa, op='within') 
            gdf_homes=gdf_homes.loc[:,['long_hogar','lat_hogar',"COD_PROV", 'NPROVINCIA','iso_country_code']]
            gdf_homes=gdf_homes.rename(columns={"COD_PROV":"id_provincia",'NPROVINCIA': 'Provincia'})
            gdf_homes=gdf_homes.dropna(subset=["id_provincia"])
        if ((pais != "CR") & (pais != "CL")):
            mapa = mapa.loc[:,["ID_1",'NAME_1','geometry']]
            gdf_homes = gpd.sjoin(gdf_homes, mapa, op='within') 
            gdf_homes=gdf_homes.loc[:,['long_hogar','lat_hogar',"ID_1",'NAME_1','iso_country_code']]
            gdf_homes=gdf_homes.rename(columns={"ID_1":"id_provincia",'NAME_1': 'Provincia'})
            gdf_homes=gdf_homes.dropna(subset=["id_provincia"])
        if pais == 'AR':
            base_nuevos=gdf_homes
        else:
            base_nuevos=base_nuevos.append(gdf_homes) 
    #df a nivel nacion   
    print(f"Asociar id_provincia a hogares: {int(time.time()-starttime)}s")
    
    
    # Creando tabla hogares final por dia
    # ==============================
    starttime = time.time()
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
        df=base_nuevos,
        dataset=True,
        table=f"base_nuevos_{codigo_fecha}",
        database="graphdata",
        dtype={'Provincia': 'string'},
        path=f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_hogares/Tablas_hogares_final/base_nuevos_{codigo_fecha}"
    )
    print(f"Guardar tabla base_nuevos_dia:{int((time.time()-starttime))}s")    
    
    
    # Creando tabla historica a nivel de hogares
    # ==============================
    starttime = time.time()
    historico_hogares = f'historico_hogares'
    tabla_hogares_fecha = f'base_nuevos_{codigo_fecha}'
    if codigo_fecha == '201': 
        # CREAR TABLA
        query = f'''CREATE TABLE {historico_hogares} 
            WITH (    
             external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/{historico_hogares}',  
             format = 'PARQUET', 
             parquet_compression = 'SNAPPY') AS 
            SELECT *
            FROM {tabla_hogares_fecha}  '''
        cursor.execute(query)
    else:
        # INSERTAR
        query = f'''INSERT INTO {historico_hogares} SELECT * FROM {tabla_hogares_fecha} '''
        cursor.execute(query) 
    print(f"Tiempo total dia {fecha}: {int(time.time() - start_time_day)}s")
    print("*"*90)
    query = (f"MSCK REPAIR TABLE historico_hogares")
    cursor.execute(query) 
    time.sleep(60)
    
    
    # Creando tabla usuarios final por dia
    # ==============================
    starttime=time.time()
    tabla_final_usuario_dia = f'usuarios_final_{codigo_fecha}'
    query = f'''CREATE TABLE {tabla_final_usuario_dia} 
         WITH (    
         external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_usuarios/Usuarios_final/{tabla_final_usuario_dia}',  
         format = 'PARQUET', 
         parquet_compression = 'SNAPPY') AS 
         SELECT filtro.caid,
                filtro.year,
                filtro.month,
                filtro.day,
                filtro.distancia_recorrida,
                filtro.n_obs,
                filtro.n_obs_noche,
                filtro.n_obs_madrugada,
                filtro.lat_hogar,
                filtro.long_hogar,
                filtro.n_obs_hogar,
                filtro.lat_mas_visitado,
                filtro.long_mas_visitado,
                filtro.n_obs_mas_visitado,
                filtro.lat_frecuente,
                filtro.long_frecuente,
                filtro.lat_final,
                filtro.long_final,
                t2.id_provincia, 
                t2.provincia,
                filtro.iso_country_code
        FROM {todos_usuarios_delta} AS filtro 
            LEFT JOIN (SELECT ROUND(lat_hogar,3) AS lat_hogar2, 
                              ROUND(long_hogar,3) AS long_hogar2, 
                              id_provincia, 
                              provincia, 
                              iso_country_code 
                       FROM historico_hogares 
                       GROUP BY ROUND(lat_hogar,3),
                                ROUND(long_hogar,3),
                                id_provincia,
                                provincia,
                                iso_country_code) AS t2  
            ON filtro.lat_final=t2.lat_hogar2 
            AND filtro.long_final=t2.long_hogar2 
            AND filtro.iso_country_code=t2.iso_country_code'''
    cursor.execute(query) 
    print(f"Crear tabla final del dia con hogares e id_provincia: {int((time.time()-starttime))}s")
    time.sleep(60)
    
    
    # Creando tabla historico a nivel de usuarios
    # ==============================
    historico_usuarios = f'historico_usuarios'
    if codigo_fecha == '201': 
        # CREAR TABLA
        query = f'''CREATE TABLE {historico_usuarios} 
            WITH (    
             external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/{historico_usuarios}',  
             format = 'PARQUET', 
             parquet_compression = 'SNAPPY',
             partitioned_by = ARRAY['iso_country_code']) AS 
            SELECT *
            FROM {tabla_final_usuario_dia} '''
        cursor.execute(query)
    else:
        # INSERTAR
        query = f'''INSERT INTO {historico_usuarios} SELECT * FROM {tabla_final_usuario_dia} '''
        cursor.execute(query) 
    print(f"Tiempo total dia {fecha}: {int(time.time() - start_time_day)}s")
    print("*"*90)

201
todos_usuarios_delta_dist_corregida_201
Tiempo query select: 11s
Tamaño tabla sin id_provincia en historico: 3170409
Pais: AR
(219306, 3)
Pais: CO
(139383, 3)
Pais: PE
(93594, 3)
Pais: NI
(0, 3)
Pais: SV
(10449, 3)
Pais: BO
(21193, 3)
Pais: DO
(42237, 3)
Pais: CL
(96071, 3)
Pais: CR
(20019, 3)
Pais: PY
(13083, 3)
Pais: BZ
(1810, 3)
Pais: HN
(17404, 3)
Pais: JM
(16815, 3)
Pais: TT
(10631, 3)
Pais: UY
(0, 3)
Pais: GY
(3199, 3)
Pais: EC
(43334, 3)
Pais: GT
(16113, 3)
Pais: PA
(9191, 3)
Pais: VE
(37484, 3)
Pais: BR
(1453757, 3)
Pais: MX
(905336, 3)
Asociar id_provincia a hogares: 191s
Guardar tabla base_nuevos_dia:2s
Tiempo total dia 2020-02-01 00:00:00: 211s
******************************************************************************************
Crear tabla final del dia con hogares e id_provincia: 7s
Tiempo total dia 2020-02-01 00:00:00: 361s
******************************************************************************************


In [15]:
query = f'''SELECT * 
            FROM {historico_hogares}
            WHERE iso_country_code = 'CL'
            LIMIT 10'''
df = cursor.execute(query).as_pandas()
df

,long_hogar,lat_hogar,id_provincia,provincia,iso_country_code
0,-70.675,-33.534,13,Región Metropolitana de Santiago,CL
1,-70.610,-33.560,13,Región Metropolitana de Santiago,CL
2,-70.523,-33.614,13,Región Metropolitana de Santiago,CL
3,-70.653,-33.545,13,Región Metropolitana de Santiago,CL
4,-70.688,-33.449,13,Región Metropolitana de Santiago,CL
5,-70.603,-33.558,13,Región Metropolitana de Santiago,CL
6,-70.584,-33.669,13,Región Metropolitana de Santiago,CL
7,-70.501,-33.359,13,Región Metropolitana de Santiago,CL
8,-70.680,-33.383,13,Región Metropolitana de Santiago,CL
9,-70.719,-33.444,13,Región Metropolitana de Santiago,CL


In [16]:
query = f'''SELECT * 
            FROM {historico_usuarios}
            WHERE iso_country_code = 'CL'
            LIMIT 10'''
df = cursor.execute(query).as_pandas()
df

,caid,year,month,day,distancia_recorrida,n_obs,n_obs_noche,n_obs_madrugada,lat_hogar,long_hogar,...,lat_mas_visitado,long_mas_visitado,n_obs_mas_visitado,lat_frecuente,long_frecuente,lat_final,long_final,id_provincia,provincia,iso_country_code
0,7333199f5e9f8b818ff0ea50cd489854e3954862f564fc...,2020,2,1,3.097831,22,0,6,-33.285,-70.886,...,-33.286,-70.887,12,-33.286,-70.887,-33.285,-70.886,13,Región Metropolitana de Santiago,CL
1,01cf1698fd5c0ca21861f160ef5397433326bb864f972c...,2020,2,1,17.018479,175,20,42,-32.992,-71.499,...,-32.992,-71.499,104,-32.992,-71.499,-32.992,-71.499,5,Región de Valparaíso,CL
2,a49431157e200221115d3d81663ab67c1de9e64961efec...,2020,2,1,0.481202,146,12,42,-33.393,-70.632,...,-33.393,-70.632,146,-33.393,-70.632,-33.393,-70.632,13,Región Metropolitana de Santiago,CL
3,3f586ddcd70b4286780d813d49ff1427322080e2b789bf...,2020,2,1,3.213820,366,20,341,-33.373,-70.515,...,-33.373,-70.515,362,NaN,NaN,-33.373,-70.515,13,Región Metropolitana de Santiago,CL
4,cfad36622c0e83c61bc2c74b7ba3c754aeee681476b931...,2020,2,1,0.051983,58,4,0,-36.977,-73.141,...,-36.977,-73.141,58,-33.451,-70.665,-36.977,-73.141,8,Región del Bío-Bío,CL
5,f3d5f3b3b7cf9cda612c290a1c394637e5f14ea7a64216...,2020,2,1,44.434244,186,6,23,-32.948,-71.540,...,-33.016,-71.553,46,-32.948,-71.540,-32.948,-71.540,5,Región de Valparaíso,CL
6,e7735cfa6faff1175acb0da92f5fee38927c07ff3c0a86...,2020,2,1,13.130798,123,48,6,-33.421,-70.608,...,-33.421,-70.610,46,NaN,NaN,-33.421,-70.608,13,Región Metropolitana de Santiago,CL
7,86583ea4abd76c8b013e49b4e11b846eb79f4bd6c47d09...,2020,2,1,0.000000,72,27,0,-37.622,-73.477,...,-37.622,-73.477,72,-37.640,-73.472,-37.622,-73.477,8,Región del Bío-Bío,CL
8,3196681deda8005ed96cfb580043f3de1b93f7192008c5...,2020,2,1,0.666518,39,0,0,NaN,NaN,...,-33.438,-71.664,27,-41.328,-72.962,-41.328,-72.962,10,Región de Los Lagos,CL
9,0f1cf23b240a7a8adf52e0ec24c850107fe12879034247...,2020,2,1,0.022527,12,8,0,-33.405,-70.676,...,-33.405,-70.676,12,-33.405,-70.676,-33.405,-70.676,13,Región Metropolitana de Santiago,CL


### Eliminando Tablas 

In [17]:
historico_hogares_deltas = f'historico_hogares_preliminar'
query = f"DROP TABLE {historico_hogares_deltas}"
cursor.execute(query)

try:
    fecha_inicio = '02-01-2020'
    fecha_final = '02-01-2020'
    fechas = pd.date_range(fecha_inicio, fecha_final)
    for fecha in fechas:
        day = str(fecha.day)
        if len(day) == 1:
            day = f'0{day}'
        codigo_fecha = str(fecha.month) + day
        print(codigo_fecha)
        
        tabla_datos_pais = f"usuarios_final_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
        tabla_datos_pais = f"base_nuevos_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
        tabla_hogares_pais = f"historico_hogares"
        query = f"DROP TABLE {tabla_hogares_pais}"
        cursor.execute(query)
        
        tabla_usuarios_pais = f"historico_usuarios"
        query = f"DROP TABLE {tabla_usuarios_pais}"
        cursor.execute(query)
        
except:
    print(f" Eliminado ")